In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect 
import numpy as np
import requests
import json

# Google API Key
from config import gkey, weatherkey

### First Store CSV data from worldcities and World Happiness Report data into DataFrames

In [16]:
# World Happiness Report data:
happiness_file = "Resources/WorldHappiness_2019.csv"
raw_happiness_data_df = pd.read_csv(happiness_file)
raw_happiness_data_df.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [17]:
# World Cities data, has 42905 cities:
cities_file = "Resources/worldcities_2022.csv"
raw_cities_data_df = pd.read_csv(cities_file)
# len(cities_data_df)
raw_cities_data_df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519


### Create new dataframes with selected columns from raw data

In [52]:
# Create a new df with cols we need from raw_happiness_data_df:
# list(happiness_data_df.columns)
happiness_data_df = raw_happiness_data_df[['Overall rank','Country or region','Score', 'Generosity', 'Perceptions of corruption']].copy()
happiness_data_df.head()

,Overall rank,Country or region,Score,Generosity,Perceptions of corruption
0,1,Finland,7.769,0.153,0.393
1,2,Denmark,7.600,0.252,0.410
2,3,Norway,7.554,0.271,0.341
3,4,Iceland,7.494,0.354,0.118
4,5,Netherlands,7.488,0.322,0.298


In [21]:
# Create a new df with cols we need from raw_cities_data_df but has all the rows (42905 cities):
# list(raw_cities_data_df.columns)
big_cities_data_df = raw_cities_data_df[['city', 'lat', 'lng', 'country', 'population']].copy()
big_cities_data_df.head()

,city,lat,lng,country,population
0,Tokyo,35.6839,139.7744,Japan,39105000.0
1,Jakarta,-6.2146,106.8451,Indonesia,35362000.0
2,Delhi,28.6667,77.2167,India,31870000.0
3,Manila,14.6000,120.9833,Philippines,23971000.0
4,São Paulo,-23.5504,-46.6339,Brazil,22495000.0


In [47]:
# Filter big_cities_data_df to only cities that are in the countries in the happiness_data_df

# First get a list of countries from happiness_df:
happiness_cntry_list = list(happiness_data_df["Country or region"])

# Then, filter big_cities_data_df by the col of "country" using the happiness_cntry_list inside isin():
cities_data_df = big_cities_data_df[big_cities_data_df["country"].isin(happiness_cntry_list)]
cities_data_df

,city,lat,lng,country,population
0,Tokyo,35.6839,139.7744,Japan,39105000.0
1,Jakarta,-6.2146,106.8451,Indonesia,35362000.0
2,Delhi,28.6667,77.2167,India,31870000.0
3,Manila,14.6000,120.9833,Philippines,23971000.0
4,São Paulo,-23.5504,-46.6339,Brazil,22495000.0
...,...,...,...,...,...
42898,Starorybnoye,72.7666,104.8000,Russia,10.0
42899,Agapa,71.4504,89.2500,Russia,10.0
42900,Tukchi,57.3670,139.5000,Russia,10.0
42901,Numto,63.6667,71.3333,Russia,10.0


In [50]:
# Next, sort cities_data_df by country and population
top10_cities_data_df = cities_data_df.sort_values(by=['country', 'population'], ascending=[True, False])
# Finally, groupby country and take the top 10 cities based on population
top10_cities_data_df = top10_cities_data_df.groupby(by=['country'], as_index=False, sort=False).head(10).reset_index(drop=True)
top10_cities_data_df

,city,lat,lng,country,population
0,Kabul,34.5328,69.1658,Afghanistan,4273156.0
1,Kandahār,31.6078,65.7053,Afghanistan,614254.0
2,Herāt,34.3738,62.1792,Afghanistan,556205.0
3,Mazār-e Sharīf,36.7000,67.1167,Afghanistan,469247.0
4,Tāluqān,36.7167,69.5167,Afghanistan,263800.0
...,...,...,...,...,...
1427,Kwekwe,-18.9167,29.9833,Zimbabwe,93072.0
1428,Kadoma,-18.3500,29.9167,Zimbabwe,79174.0
1429,Masvingo,-20.0744,30.8328,Zimbabwe,72527.0
1430,Chinhoyi,-17.3500,30.2000,Zimbabwe,56794.0


### Store JSON data into a DataFrame

In [4]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,id,address,longitude,latitude,us_state
0,1,043 Mockingbird Place,-86.5186,39.1682,Indiana
1,2,4 Prentice Point,-85.0707,41.0938,Indiana
2,3,46 Derek Junction,-96.7776,32.7673,Texas
3,4,11966 Old Shore Place,-94.3567,39.0350,Missouri
4,5,5 Evergreen Circle,-73.9772,40.7808,New York


### Clean DataFrame

In [5]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database

In [6]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5433
database_name = 'customer_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Check for tables

In [16]:
# Old way will be deprecated:
# engine.table_names()
# New way:
inspector = inspect(engine)
inspector.get_table_names() 

/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['customer_name', 'customer_location']

### Use pandas to load csv converted DataFrame into database

In [17]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [18]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [19]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [20]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
